<a href="https://colab.research.google.com/github/Lucas-Melo-A-S/Data_Science/blob/main/IA_av_qualis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AVqualis - Inteligencia Artificial

O presente trabalho tem como objetivo identificar qual tipo de fruta é atravês das imagens importadas no sistema. Esse é um projeto comum na comunidade de tecnologia.
O projeto a seguir foi baseado no projeto realizado pelo Cientista de Dados Gabriel Atkin.

Para inicio do projeto iremos importar as bibliotecas que serão utilizadas aos longo do nosso projeto. Vale informar que também podemos importar as bibliotecas/ metodos ao longo de todo o corpo do projeto mas para melhor organização iremos deixar estas no inicio.

In [ ]:
#Importando as Bibliotecas:

import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img,img_to_array
print(tf.__version__)

2.8.0


- Criando a lista com o filepath para treinamento e teste -  Iremos utilizar o metodo Path para que não tenhamos erros se caso tenha mudança de SO. Como o Windows tem caminhos(path) diferente do OS(MAC),Linux podemos utilizar o Path como um opção, pois esse método irá escrever o arquivo de acordo com o SO utilizado.

- Com o  metodo GLOB iremos falar para o sistema pesquisar ao longo de todo o arquivo procurando arquivo que ser sufixoé .jpg. Com isso iremos selecionar apenas imagens.


In [ ]:
#1
image_dir = Path("C:\\Users\\User\\Downloads\\archive\\images")

filepaths = list(image_dir.glob(r'**/*.jpg'))


In [ ]:
list(image_dir.glob('**/*.jpg'))

[]

- Para a criação do DataFrame concatenar nosso filepath com a sua classe.
 - Para encontrar marcar a classe iremos procurar no proprio diretorio do arquivo. Para isso iremos utilizar o metodo SPLIT() para realizarmos o corte do arquivo até deixarmos apenas a classe de cada arquivo. Para automatizar esse processo iremos fazer um for em todo o diretorio dos arquivos(filepath)

- Encontrado a classe, iremos trnasformar o datapath e os labels em pandas dataframe e depois iremos concatenar tudo em dentro um novo dataframe chamado df.
 - Para que as classes não fiquem sempre uma seguida da outra, fazemos uma mistura(shuffle) e resetando também seu index para que o index fique de acordo com o novo dataframe criado

In [ ]:
    # Create a DataFrame with the filepath and the labels of the pictures

    # You you probably need to adapt this split method to your Pc path, mine was using this double bars
# labels = [str(filepaths[i]).split("\\")[-2] \
#             for i in range(len(filepaths))]
    
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepath = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

    # Concatenate filepaths and labels
images = pd.concat([filepath, labels], axis=1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
images

,Filepath,Label


Como podemos ver nessa situação teremos 101000 imagens e com isso seria muito demorado fazer a classificação de todas as imagens. Com isso iremos reduzir a quantidade de imagens para que nosso processo seja mais rápido.


In [ ]:
category_samples = []
for category in images['Label'].unique():
    category_slice = images.query("Label == @category")
    category_samples.append(category_slice.sample(100, random_state=1))
image_df = pd.concat(category_samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)

ValueError: ignored

Após essa separação o nosso dataframe terá 10100 imagens, sendo assim o processo ficará mais rápido.

In [ ]:
image_df

In [ ]:
image_df['Labels'].value_counts()

#Treinando e Testando 

Após todo o processo de modelagem e limpeza do dataframe, iremos fazer o teste e treino.
- Utilizaremos o metodo train_test_split, onde nesse iremos dividir o dataframe em dois, o de teste e o de treino, nesse método temos que falar o dataframe que será recortado, o tamanho do train/test.(Nessa situação também falamos para misturar pois não queremos que o teste ou o treino contenham somente um tipo de classe, também é utilizado o randon_state para que sempre que realizarmos esse ação, os dados divididos sejam os mesmos.)

In [ ]:
#Separando em dataframe de teste e treino
train_df,test_df = train_test_split(image_df,train_size = 0.7, 
                                    shuffle = True, 
                                    random_state = 2)


NameError: ignored

Geradores

- Iremos utilar a biblioteca TensorFlow para carregar as imagens uma de cada vez para não sobrecarregar a memoria. Com os geradores iremos especificar como que os dados irão carregar durante o treinamento.

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split = 0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input,
)

Especificando os geradores de como queremos carregar os dados durante o treino.
- Nessa situação iremos utilizar o método 'flow_from_dataframe' onde iremos especificar a imagem  que estamos usando para que o gerador pegue essas imagens do dataframe para treina-las e assim recicla-las.
- Nesse metodo teremos que especificar o dataframe, a coluna em X e Y, dando o tamanho da imagem, cor, o modo da classe entre outras funções.Esse metodo vai ser utilizado tanto para o dataframe de treino(train), quanto para o de teste(test). No caso da validação, iremos criar uma variavel, onde esta variavel irá receber o train_generator.(Preste atenção no subset, ele modificação).


In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'Filepath',
    y_col = 'Label',
    target.size = (224,224),
    color_mode = 'rgb',
    classe_mode = 'categorical',
    batch_size = 32,
    shuffle = True,
    seed = 42,
    subset = 'training' #Utilizado quando temos a validação no gerador.

)


val_images= train_generator.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'Filepath',
    y_col = 'Label',
    target.size = (224,224),
    color_mode = 'rgb',
    classe_mode = 'categorical',
    batch_size = 32,
    shuffle = True,
    seed = 42,
    subset = 'validation'
)




test_images = test_generator.flow_from_dataframe(
    dataframe = test_df,
    x_col = 'Filepath',
    y_col = 'Label',
    target.size = (224,224),
    color_mode = 'rgb',
    classe_mode = 'categorical',
    batch_size = 32,
    shuffle = False,
    seed = 42,
)

#Modelagem

Para a modelagem iremos utilizar o MobileNetV2, esse método retorna um modelo de classificação de Imagem Keras.

In [ ]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape = (224,224,3),
    include_top = False,
    weights = 'imagenet',
    pooling = 'avg'
)

pretrained_model.trainable = False #Esse comodo faz termos certeza que não iremos modificar o peso das imagens no dataframe original.

In [ ]:
# Training the model take some minutes, so if you don't want to run this cell its ok, the trained model for vegetables and fruits is already saved in the project root(FV.h5)
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(101, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

print(model.summary())


# Treino

Para o treinamento iremos primeiro compilar o input com output e após isso iremos fazer o treinamento do modelo com o método .fit.

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    batch_size = 32,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=2,
            restore_best_weights=True
        )
    ]
)

# Resultados

In [ ]:
results = model.evaluate(test_images, verbose = 0)
print("Test Accuracy:{:.2f}".format(results[1] * 100))

In [ ]:
# Predict the label of the test_images
pred = np.argmax(model.predict(test_images), axis = 1)



# Map the label
labels = (train_images.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred1 = [labels[k] for k in pred]
pred1

In [ ]:
#Confusion Matrix
cm = confusion_matrix(test_images.labels,pred)
clr = classification_report(test_images.labels, predictions, target_names = test.images.class_indices)

plt.figura(figsize = (30, 30))
sns.heatmap(cm, annot = True, fmt = 'g', cmap = 'Blues', cbar = False)

plt.xtick(ticks = np.arange(101) + 0.5, labels = test_images.class_indices,rotatio = 90)
plt.ytick(ticks = np.arange(101) + 0.5, labels = test_images.class_indices,rotatio = 0)
plt.xlabel('pREDICTED')
plt.title('Confusion Matrix')

In [ ]:
print("Classification Report:\n-------------\n", clr)

In [ ]:
test_images.class_indices

In [ ]:
def output(location):
    img=load_img(location,target_size=(224,224,3))
    img=img_to_array(img)
    img=img/255
    img=np.expand_dims(img,[0])
    answer=model.predict(img)
    y_class = answer.argmax(axis=-1)
    y = " ".join(str(x) for x in y_class)
    y = int(y)
    res = labels[y]
    return res

In [ ]:
img = output('../Fruit_Vegetable_Recognition/Dataset/train/cabbage/Image_1.jpg')
img

In [ ]:
model.save('FV.h5')